In [8]:
import pandas as pd

df = pd.read_csv('fil_data_movies_lessthan10k_removed.csv')
df = df.drop(columns=['id', 'title','release_date',  'adult','original_title', 'backdrop_path', 'homepage', 'imdb_id', 'overview', 'poster_path', 'tagline','spoken_languages' ])
df.head()

,vote_average,vote_count,revenue,runtime,budget,original_language,popularity,genres,production_companies,production_countries
0,8.364,34495,825532764,148,160000000,en,83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America"
1,8.417,32571,701729206,169,165000000,en,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America"
2,8.512,30619,1004558444,152,185000000,en,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America"
3,7.573,29815,2923706026,162,237000000,en,79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom"
4,7.710,29166,1518815515,143,220000000,en,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America


In [2]:
df_encoded = pd.get_dummies(df['genres'].str.split(', ').explode()).groupby(level=0).sum()
#print(df_encoded.head())

df_encoded2 = pd.get_dummies(df['original_language'].str.split(', ').explode()).groupby(level=0).sum()
df_new = pd.DataFrame()
df_new['en'] = df_encoded2['en']
df_new['other'] = df_encoded2.drop('en', axis=1).sum(axis=1)
#print(df_new)

df_encoded3 = pd.get_dummies(df['production_companies'].str.split(', ').explode()).groupby(level=0).sum()
mask = (df_encoded3.sum() >= 180)
df_encoded3 = df_encoded3.loc[:, mask.values]
#print(df_encoded3.sum())

df_encoded4 = pd.get_dummies(df['production_countries'].str.split(', ').explode()).groupby(level=0).sum()
df_new2 = pd.DataFrame()
df_new2['United States of America'] = df_encoded4['United States of America']
df_new2['United Kingdom'] = df_encoded4['United Kingdom']
df_new2['other2'] = df_encoded4.drop(['United States of America', 'United Kingdom'], axis=1).sum(axis=1)
#print(df_new2.sum())


In [3]:
df = pd.concat([df, df_encoded], axis=1)
df = df.drop(columns=['genres'])

df = pd.concat([df, df_new], axis=1)
df = df.drop(columns=['original_language'])

df = pd.concat([df, df_encoded3], axis=1)
df = df.drop(columns=['production_companies'])

df = pd.concat([df, df_new2], axis=1)
df = df.drop(columns=['production_countries'])

df.head()

,vote_average,vote_count,revenue,runtime,budget,popularity,Action,Adventure,Animation,Comedy,...,Columbia Pictures,Metro-Goldwyn-Mayer,New Line Cinema,Paramount,Universal Pictures,Walt Disney Pictures,Warner Bros. Pictures,United States of America,United Kingdom,other2
0,8.364,34495,825532764,148,160000000,83.952,1,1,0,0,...,0,0,0,0,0,0,1,1,1,0
1,8.417,32571,701729206,169,165000000,140.241,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
2,8.512,30619,1004558444,152,185000000,130.643,1,0,0,0,...,0,0,0,0,0,0,1,1,1,0
3,7.573,29815,2923706026,162,237000000,79.932,1,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4,7.710,29166,1518815515,143,220000000,98.082,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Define the features and the target
X = df.drop('vote_average', axis=1)
y = df['vote_average']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)


In [5]:
# Import metrics
from sklearn import metrics

# Make predictions on the entire data set
y_pred_all = model.predict(X_test)

# Compare the predicted values with the actual values
df_scores_comp_all = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_all})

# Print the comparison dataframe
#print(df_scores_comp_all)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Create a list of models
models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor()]

# Define the hyperparameters for each model
hyperparameters = {
    'LinearRegression': {},
    'DecisionTreeRegressor': {'max_depth': [None, 5, 10]},
    'RandomForestRegressor': {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10]}
}

for model in models:
    model_name = model.__class__.__name__
    
    # Initialize the grid search
    grid_search = GridSearchCV(model, hyperparameters[model_name])
    
    # Train the model with the grid search
    grid_search.fit(X_train, y_train)
    
    # Make predictions with the best found model
    y_pred = grid_search.best_estimator_.predict(X_test)
    
    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    r2_percentage = r2 * 100
    print(f"Model: {model_name}, Best Hyperparameters: {grid_search.best_params_}, R-squared (%): {r2_percentage}")



Model: LinearRegression, Best Hyperparameters: {}, R-squared (%): 41.50224618316637
Model: DecisionTreeRegressor, Best Hyperparameters: {'max_depth': 5}, R-squared (%): 33.169912048567255
Model: RandomForestRegressor, Best Hyperparameters: {'max_depth': None, 'n_estimators': 300}, R-squared (%): 53.60889732815792


In [7]:
from sklearn.ensemble import RandomForestRegressor

best_params_ = grid_search.best_params_
model = RandomForestRegressor(**best_params_)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred)*100)

53.838018162441195
